In [1]:
import numpy as np
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

/home/key/software/anaconda3/envs/tf3.5/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
mnist = tf.contrib.learn.datasets.load_dataset("mnist")
train_data = mnist.train.images # Returns np.array
train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
eval_data = mnist.test.images # Returns np.array
eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

Extracting MNIST-data/train-images-idx3-ubyte.gz
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz


Unlike in raw tensorflow, with the Estimators API we specify the number of neurons in a layer directly, not indirectly via a weights matrix. <br />
The distinction of defining and running the graph is also less obvious (we never create a session).

In [3]:
def cnn_model_fn(features, labels, mode):
  """Model function for CNN."""
  # Input Layer
  input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

  # Convolutional Layer #1
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #1
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2 and Pooling Layer #2
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Dense Layer
  pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
  dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits Layer
  logits = tf.layers.dense(inputs=dropout, units=10)

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
  loss = tf.losses.softmax_cross_entropy(
      onehot_labels=onehot_labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)


In [4]:
mnist_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn, model_dir="/tmp/mnist_convnet_model")


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_tf_random_seed': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f65ae29dba8>, '_keep_checkpoint_max': 5, '_model_dir': '/tmp/mnist_convnet_model', '_save_checkpoints_steps': None, '_master': '', '_num_worker_replicas': 1, '_num_ps_replicas': 0, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_is_chief': True, '_session_config': None, '_save_checkpoints_secs': 600, '_save_summary_steps': 100, '_task_type': 'worker', '_log_step_count_steps': 100, '_task_id': 0}


In [5]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},
    y=train_labels,
    batch_size=100,
    num_epochs=None,
    shuffle=True)

In [6]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data},
    y=eval_labels,
    num_epochs=1,
    shuffle=False)

In [7]:
mnist_classifier.train(
    input_fn=train_input_fn,
    steps=1000)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/mnist_convnet_model/model.ckpt.
INFO:tensorflow:step = 1, loss = 2.2985456
INFO:tensorflow:global_step/sec: 3.4226
INFO:tensorflow:step = 101, loss = 2.2855976 (29.219 sec)
INFO:tensorflow:global_step/sec: 3.37228
INFO:tensorflow:step = 201, loss = 2.2761366 (29.661 sec)
INFO:tensorflow:global_step/sec: 3.33024
INFO:tensorflow:step = 301, loss = 2.2538326 (30.021 sec)
INFO:tensorflow:global_step/sec: 3.46439
INFO:tensorflow:step = 401, loss = 2.2603416 (28.865 sec)
INFO:tensorflow:global_step/sec: 3.34219
INFO:tensorflow:step = 501, loss = 2.2426932 (29.920 sec)
INFO:tensorflow:global_step/sec: 3.4574
INFO:tensorflow:step = 601, loss = 2.2261662 (28.923 sec)
INFO:tensorflow:global_step/sec: 3.48873
INFO:ten

In [8]:
eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

INFO:tensorflow:Starting evaluation at 2017-12-20-18:26:41
INFO:tensorflow:Restoring parameters from /tmp/mnist_convnet_model/model.ckpt-1000
INFO:tensorflow:Finished evaluation at 2017-12-20-18:26:51
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.7227, global_step = 1000, loss = 1.76225
{'accuracy': 0.7227, 'loss': 1.7622501, 'global_step': 1000}
